# Import CAMD-XRD Package - hsg and relevant packages

In [8]:
from tqdm.notebook import tqdm
import hsg
import numpy as np
from pymatgen.analysis.diffraction import xrd

# Relevant Parameters from Unsolved LiFeO$_2$ Pattern
## Ref: Sakurai et al. J. Power Sources 1997


In [2]:
#Li3CrO4
a = 9.677
b = 2.934
c = 5.005
alpha = 90
beta = 90
gamma = 90
d_tol = 1.5
d_mins = {}
hsgtests = []
spgs = [62]

atoms = [(4, 'Li'), (4, 'Fe'), (8, 'O')]
lifeo2twothetas = [19.945, 25.569, 32.989,35.408, 35.728,36.634,37.057,40.151,40.528,45.473,48.678,50.493,51.501,52.228,52.551,55.990,58.436,60.110,60.370,63.298,64.127,66.016]
lifeo2intensities = [100,33,38,8,43,23,13,24,12,34,9,3,8,10,13,12,9,18,15,8,12,6]
         
strucs_new_full = []

for i in range(len(spgs)):
    density = 0.5
    
    hsgtest = hsg.HierarchicalStructureGeneration(spgs[i], a,b,c, alpha, beta, gamma, atoms, d_tol, d_mins, \
                                                         use_asu = True)

    for k in tqdm(range(len(hsgtest.filter_combinations))):
        strucs_new=hsgtest.get_structure_grid(density,combination = k)
        strucs_new_full.append([])
        for struc in tqdm(strucs_new):
            strucs_new_full[-1].append(hsgtest.get_pymatgen_structure(struc))

# Calculate XRD patterns using Pymatgen

In [4]:
matpatterns_all = []
xrd_calc = xrd.XRDCalculator()
for strucs_spg in tqdm(strucs_new_full):
    patterns = []
    for strucs in strucs_spg:
        patterns.append(xrd_calc.get_pattern(strucs))
    matpatterns_all.append(patterns)


# Calculate R values

In [5]:
Rvalues = []
for j in tqdm(range(len(matpatterns_all))):
    Rvalues.append(hsg.get_Rs(lifeo2twothetas, lifeo2intensities, matpatterns_all[j]))

# Determine lowest R value structures for DFT relaxation

In [9]:
lowests = []
struc_indexes = []
lowest = []
for i in range(len(Rvalues)):
    for k in range(len(Rvalues[i])):

        if len(lowest) == 100:
            order = np.argsort(lowest)
            if Rvalues[i][k] < lowest[order[-1]]:
                lowest[order[-1]] = Rvalues[i][k]
                struc_indexes[order[-1]] = (i,k)
        else:
            lowest.append(Rvalues[i][k])
            struc_indexes.append((i,k))

test_strucs = []

for index2 in struc_indexes:
    test_strucs.append(strucs_new_full[index2[0]][index2[1]])
